In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [14]:
import braindecode
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# load nn model from braindecode
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.deep4 import Deep4Net
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

import core.data as data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# load data 
x_loaded, y_loaded = data.load_x('data/x_train.h5'), data.load_y('data/y_train.csv')

Started loading file data/x_train.h5
Finished loading the file.
Started loading file data/y_train.csv
Finished loading the file.


In [4]:
def class_weights(y):
    class_sample_count = np.array(\
                [len(np.where(y == t)[0]) for t in range(2)])
    return float(len(y)) / class_sample_count


In [5]:
# convert y to categorical with np.eye(d)[y_loaded]
# and flatten the 40 independent samples
x, y = data.flatten_x(x_loaded), data.flatten_y(y_loaded, repeat=40)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.66)
x_train, x_test = x_train.squeeze(), x_test.squeeze()

# Only one value : 0 or 1  
y_train, y_test = np.argmax(y_train, axis=1), np.argmax(y_test, axis=1)

# get class weights
weights = class_weights(y_train)

print('shape : ', x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print('weights :', weights)

shape :  (24974, 7, 500) (12866, 7, 500) (24974,) (12866,)
weights : [1.28262544 4.53825186]


In [18]:
# define the model

cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = x_train.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = Deep4Net(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=x_train.shape[2],
                        final_conv_length='auto') 

from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
criterion = lambda prediction, targets : F.nll_loss(prediction, targets, weight=torch.from_numpy(weights).float())
model.compile(loss=criterion, optimizer=optimizer, iterator_seed=1,)

In [19]:
model.fit(x_train, y_train, epochs=5, batch_size=64, scheduler='cosine'
         ,validation_data=(x_test, y_test)
         ,input_time_length = 450) # supercropsize for cropped training
# Rk : here, 1 timestep = 1 / 250 seconds

2020-02-08 17:00:05,030 INFO : Run until first stop...
2020-02-08 17:00:31,926 INFO : Epoch 0
2020-02-08 17:00:31,927 INFO : train_loss                20.75311
2020-02-08 17:00:31,927 INFO : valid_loss                19.00763
2020-02-08 17:00:31,927 INFO : train_misclass            0.24782
2020-02-08 17:00:31,931 INFO : valid_misclass            0.25019
2020-02-08 17:00:31,932 INFO : runtime                   0.00000
2020-02-08 17:00:31,933 INFO : 
2020-02-08 17:02:01,765 INFO : Time only for training updates: 89.83s
2020-02-08 17:02:38,019 INFO : Epoch 1
2020-02-08 17:02:38,020 INFO : train_loss                0.70720
2020-02-08 17:02:38,022 INFO : valid_loss                0.70786
2020-02-08 17:02:38,031 INFO : train_misclass            0.22872
2020-02-08 17:02:38,032 INFO : valid_misclass            0.23154
2020-02-08 17:02:38,033 INFO : runtime                   116.73493
2020-02-08 17:02:38,034 INFO : 
2020-02-08 17:04:20,383 INFO : Time only for training updates: 102.35s
2020-02-

In [39]:
x_challenge = data.load_x('data/x_test.h5')
x_challenge = data.flatten_x(x_challenge).squeeze()

Started loading file data/x_test.h5
Finished loading the file.


In [40]:
# average the predictions of all the indepedent trials to return one classification for each subject
def average_predictions(predictions, nb_trials = 40):
    # number of samples
    n = int(len(predictions) / nb_trials)
    avg_preds = np.zeros(n)
    for i in range(n):
        sample_preds = predictions[i*nb_trials:(i+1)*nb_trials]
        avg_preds[i] = int(np.mean(sample_preds) > 0.5)
    return avg_preds

In [47]:
y_challenge = model.predict_classes(x_challenge)
y_challenge2 = average_predictions(y_challenge)

In [51]:
import csv

def save_csv(y, file_name):
    with open(file_name, 'w') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['id', 'label'])
        for i in range(len(y)):
            writer.writerow([str(i), str(int(y[i]))])

In [52]:
save_csv(y_challenge2, 'data/result.csv')